In [63]:
# Auto Reload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Import Modules
import numpy as np
import copy
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from pprint import pprint

In [62]:
# Import Custom Modules
from helpers import *

In [4]:
# # 2019 Info
# top_songs_2019 = get_top_songs(2019)

# # 2018 Info
# top_songs_2018 = get_top_songs(2018)

# # 2017 Info
# top_songs_2017 = get_top_songs(2017)

# # 2016 Info
# top_songs_2016 = get_top_songs(2016)

# # 2015 Info
# top_songs_2015 = get_top_songs(2015)

# # Year list
# years = [2015, 2016, 2017, 2018, 2019]
# # List of top song tuples by year
# top_songs_list = [top_songs_2015, top_songs_2016, top_songs_2017, top_songs_2018, top_songs_2019]

In [5]:
years = list(range(2000, 2020))
top_songs = [get_top_songs(year) for year in years]

In [6]:
# Create DataFrame
df = pd.DataFrame(
        columns=['song', 'artist', 'featured', 'rank', 'year'],
)

In [7]:
# # Populate Dataframe
# for year, info in zip(years, top_songs_list):
#     x = pd.DataFrame(
#         data=[[*info[x], year] for x in range(len(info))],
#         columns=['song', 'artist', 'featured', 'rank', 'year'])
#     df = df.append(x, ignore_index=True)


In [8]:
# Populate Dataframe
for year, info in zip(years, top_songs):
    x = pd.DataFrame(
        data=[[*info[x], year] for x in range(len(info))],
        columns=['song', 'artist', 'featured', 'rank', 'year'])
    df = df.append(x, ignore_index=True)


In [12]:
# df.info()
df.head()

,song,artist,featured,rank,year
0,Thrift Shop,Macklemore & Ryan Lewis,,1,2000
1,Can't Hold Us,Macklemore & Ryan Lewis,,2,2000
2,Holy Grail,Jay Z,,3,2000
3,Started From The Bottom,Drake,,4,2000
4,Feel This Moment,Pitbull,,5,2000


In [10]:
responses, lyrics = [], []
for song, artist in zip(df['song'], df['artist']):
    res = ""
    try:
        response = get_extract_response(song, artist)
        if not response:
            response = get_extract_response(song, '')
        res = scrape_song_lyrics_from_url(response)
    except:
        print(f"Something happened with {song}: {artist}")
        pass
    responses.append(response)
    lyrics.append(res)

In [49]:
# # len(responses), len(lyrics)
# lyric_state_err = []
# song_id_err = []
# lyric_owner_id_err = []
# prime_art_url_err = []

# for i in range(len(responses)):
#     err = -1
#     try:
#         for info in ['lyrics_state', 'id', 'lyrics_owner_id', 'primary_artist']:
#             responses[i]['result'][info]
#     except:
#         d_map = dict(zip(['lyrics_state', 'id', 'lyrics_owner_id', 'primary_artist'], [lyric_state_err, song_id_err, lyric_owner_id_err, prime_art_url_err]))
#         print(f"Error at {i} {info}")
#         d_map[info].append(i)
        
# for x in [lyric_state_err, song_id_err, lyric_owner_id_err, prime_art_url_err]:
#     print(x)
    
responses[0]['result']
# for x in range(len(responses)):
#     try:
#         responses[x]['result']['primary_artist']
#     except:
#         print(x)

{'_type': 'song',
 'annotation_count': 23,
 'api_path': '/songs/86538',
 'full_title': 'Thrift Shop by\xa0Macklemore\xa0& Ryan Lewis (Ft.\xa0Wanz)',
 'header_image_thumbnail_url': 'https://images.genius.com/b6b7c8297c29451e9063051259f76a3e.300x300x1.jpg',
 'header_image_url': 'https://images.genius.com/b6b7c8297c29451e9063051259f76a3e.578x578x1.jpg',
 'id': 86538,
 'instrumental': False,
 'lyrics_owner_id': 3928,
 'lyrics_state': 'complete',
 'lyrics_updated_at': 1584671915,
 'path': '/Macklemore-and-ryan-lewis-thrift-shop-lyrics',
 'pyongs_count': 164,
 'song_art_image_thumbnail_url': 'https://images.rapgenius.com/777e19d177532620387a32f68148cb0d.300x300x1.jpg',
 'song_art_image_url': 'https://images.rapgenius.com/777e19d177532620387a32f68148cb0d.700x700x1.jpg',
 'stats': {'unreviewed_annotations': 0,
  'concurrents': 2,
  'hot': False,
  'pageviews': 4505342},
 'title': 'Thrift Shop',
 'title_with_featured': 'Thrift Shop (Ft.\xa0Wanz)',
 'updated_by_human_at': 1584985184,
 'url': 'ht

In [66]:
# Create Series for each feature
lyrics_state = pd.Series([response['result']['lyrics_state'] == 'complete' if 'lyrics_state' in response['result'] else True for response in responses]) 
song_id =  pd.Series([response['result']['id'] for response in responses])
lyrics_owner_id = pd.Series([response['result']['lyrics_owner_id'] if 'lyrics_owner_id' in response['result'] else -1 for response in responses])
primary_artist_url = pd.Series([response['result']['primary_artist']['url'] if 'lyrics_owner_id' in response['result'] else '' for response in responses])

In [69]:
# Create More Features for Each Song ['lyrics_state', 'id', 'lyric_owner_id']

# Match Lyrics To Song
df['lyrics'] = pd.Series(lyrics)

# Match Lyric State To Song
df['lyrics_state'] = pd.Series(lyrics_state)

# Match song id To Song
df['song_id'] = pd.Series(song_id)

# Match lyric owner To Song
df['lyrics_owner_id'] = pd.Series(lyrics_owner_id)

# Match Artist Url To Song
df['primary_artist_url'] = pd.Series(primary_artist_url)



In [70]:
df

,song,artist,featured,rank,year,lyrics,lyrics_state,song_id,lyrics_owner_id,primary_artist_url
0,Thrift Shop,Macklemore & Ryan Lewis,,1,2000,"""Hey, Macklemore, can we go thrift shopping?""\...",True,86538,3928,https://genius.com/artists/Macklemore-and-ryan...
1,Can't Hold Us,Macklemore & Ryan Lewis,,2,2000,"Hey, hey, hey\nGood to see you\nCome on, dude,...",True,57234,37383,https://genius.com/artists/Macklemore-and-ryan...
2,Holy Grail,Jay Z,,3,2000,You'd take the clothes off my back and I'd let...,True,177832,104344,https://genius.com/artists/Jay-z
3,Started From The Bottom,Drake,,4,2000,Started…\nZombie on the track\n\nStarted from ...,True,113898,103043,https://genius.com/artists/Drake
4,Feel This Moment,Pitbull,,5,2000,"Ask for money, and get advice\nAsk for advice,...",True,115843,174558,https://genius.com/artists/Pitbull
...,...,...,...,...,...,...,...,...,...,...
995,Press,Cardi B,,46,2019,"Monsta's gon' tear it up\nBardi\nWoo, yeah\nBi...",True,4191420,8117646,https://genius.com/artists/Cardi-b
996,Backin' It Up,Pardison Fontaine,,47,2019,"Ahhh\nCardi!\nTurn around, fuck it all the way...",True,3970189,104344,https://genius.com/artists/Pardison-fontaine
997,Twerk,City Girls,,48,2019,"(I want a slim, fine woman with some twerk wit...",True,4080406,1603328,https://genius.com/artists/City-girls
998,Time,NF,,49,2019,Even if we both break down tonight\nAnd you sa...,True,4693540,3375166,https://genius.com/artists/Nf


In [73]:
# Fix F**kin Problems by A$AP Rocky
df['lyrics'].fillna(problem_lyrics, inplace=True)
# df[df['lyrics'].isna()]

In [ ]:
# df.to_csv('data.csv')

In [74]:
df.to_csv('all_data.csv')